### IObservable&lt;T&gt; &  IObserver&lt;T&gt; interfaces

>  The **IObservable<T>** interface is defined in the System namespace in C#.
>    - It provides a mechanism for representing a stream of data that can be observed by multiple observers.
>  
>    - When a class implements the *IObservable<T>* interface, it must provide an implementation for the *Subscribe(IObserver<T>)* method.
>    - This method is called when an observer subscribes to the observable, and it should return an object that represents the subscription.
>
>
>  The **IObserver<T>** interface is also defined in the System namespace in C#.
>    - It provides a mechanism for receiving notifications from an observable.  
>
>    - When a class implements the IObserver<T> interface, it must provide an implementation for the following methods:  
>       - OnNext(T): This method is called when the observable produces a new value.  
>       - OnError(Exception): This method is called when the observable encounters an error. 
>       - OnCompleted(): This method is called when the observable has completed producing values.  
>
>Here is an example of how to use the IObservable<T> interface:

In [ ]:
using System;

public class StockPriceTracker : IObservable<decimal>
{
    private List<IObserver<decimal>> observers = new List<IObserver<decimal>>();
    private readonly Random _random = new Random();
    private readonly decimal _initialPrice;
    private  decimal stockPrice;
    private readonly decimal _maxPriceChange;
    private readonly decimal _minPriceChange;


 // The StockPrice property notifies the observers whenever it changes.
    public decimal StockPrice
    {
        get { return stockPrice; }
        set
        {
            stockPrice = value;
            NotifyObservers();

        }
    }
    public StockPriceTracker(decimal initialPrice, decimal maxPriceChange, decimal minPriceChange)
    {
        _initialPrice = initialPrice;
        _maxPriceChange = maxPriceChange;
        _minPriceChange = minPriceChange;
    }

    public IDisposable Subscribe(IObserver<decimal> observer)
    {
         if (!observers.Contains(observer))
        {
            observers.Add(observer);
        }
       
         return new Unsubscriber(observers, observer);
    }
    public void GetStockPrices()  {
         decimal currentPrice = _initialPrice;


        for (int i=0;i<50;i++)
        {
            var newPrice =  currentPrice + (decimal)(_random.NextDouble() * (double)(_maxPriceChange - _minPriceChange) + (double)_minPriceChange);
            StockPrice = newPrice;
        }
    }

     // NotifyObservers notifies all the observers in the list of observers.
    private void NotifyObservers()
    {
        foreach (var observer in observers)
        {
            observer.OnNext(stockPrice);
        }
    }
        // Unsubscriber is a helper class that implements the IDisposable interface.
    // It removes an observer from the list of observers when it is disposed.
    private class Unsubscriber : IDisposable
    {
        private List<IObserver<decimal>> _observers;
        private IObserver<decimal> _observer;

        public Unsubscriber(List<IObserver<decimal>> observers, IObserver<decimal> observer)
        {
            _observers = observers;
            _observer = observer;
        }

        public void Dispose()
        {
            if (_observer != null && _observers.Contains(_observer))
            {
                _observers.Remove(_observer);
            }
        }
    }
}


public class StockPriceDisplay : IObserver<decimal>
{
    private readonly string _name;

    public StockPriceDisplay(string name)
    {
        _name = name;
    }

    public void OnCompleted()
    {
        Console.WriteLine("The stock price tracker has completed transmitting data to {0}.", _name);
    }

    public void OnError(Exception error)
    {
        Console.WriteLine("{0}: The stock price tracker cannot be reached.", _name);
    }

    public void OnNext(decimal value)
    {
        Console.WriteLine("{0}: The current stock price is {1:C}.", _name, value);
    }
}

//Usage 
        StockPriceTracker stockPriceTracker = new StockPriceTracker(100.00m, 10.00m, -10.00m);
        StockPriceDisplay stockPriceDisplay1 = new StockPriceDisplay("Display 1");
        StockPriceDisplay stockPriceDisplay2 = new StockPriceDisplay("Display 2");

        IDisposable subscription1 = stockPriceTracker.Subscribe(stockPriceDisplay1);
        IDisposable subscription2 = stockPriceTracker.Subscribe(stockPriceDisplay2);

        stockPriceTracker.GetStockPrices();
        subscription1.Dispose();
        subscription2.Dispose();
    

# Continue learning

There are plenty more resources out there to learn!

> [⏩ Next Module - ASync Pragramming ](../C#5.0/29.ASyncPragramming.ipynb)
>
> [⏪ Last Module - Generic Co and Contravariance](28.GenericCoAndContravariance.ipynb)
>
> [Reference : Covariance and Contravariance](https://learn.microsoft.com/en-us/dotnet/standard/generics/covariance-and-contravariance)